<a href="https://colab.research.google.com/github/YoshiyukiKono/pulsar_for_beginners/blob/main/01_pulsar_client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pulsar クライアント開発演習

## 演習: 第一部 Pulsar データ入力とAstra DBへのSink

## 概要

### 事前準備
 - 環境準備
  - Astraアカウント登録・サインイン
  - Astra DB データベース、キースペース作成
  - Astra Streaming テナント作成

 - 開発準備
  - Astra DB テーブル作成
  - Astra Streaming ネームスペース、トピック、サブスクリプション作成
  - Astra DB トークン生成
  - Astra Streaming トークン生成
 - データベース連携
  - Astra Streaming Sink作成

### 演習概要

 - Pulsarクライアントライブラリインストール
 - Pulsar接続設定
 - Pulsarプロデューサー実行
 - Pulsarコンシューマー実行
 - Astra DBへのSink結果の確認
---


## 演習


### Pulsarクライアントライブラリインストール

In [2]:
!pip install pulsar-client==3.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 37.4 MB/s eta 0:00:00


### Astra Streaming 接続設定

In [11]:
service_url = input("service url:")


service url:pulsar+ssl://pulsar-gcp-useast1.streaming.datastax.com:6651


In [6]:
import getpass

token = getpass.getpass()
print("token:" + token[0:5] + "..." + token[-5:])
print("token length:" + str(len(token)))

··········
token:eyJhb...euW8Q
token length:548


In [7]:
your_tenant_name = input("tenant name:")

tenant name:cdc


In [8]:
your_namespace = input("namespace:")
your_topic_name = input("topic name:")
topic_url = 'persistent://' + your_tenant_name + '/' + your_namespace + '/' + your_topic_name
print("topic url:" + topic_url)

namespace:default
topic name:astrasink
topic url:persistent://cdc/default/astrasink


In [12]:
import pulsar

client = pulsar.Client(service_url,
                        authentication=pulsar.AuthenticationToken(token))
topic_partitions = client.get_topic_partitions(topic_url)
print(topic_partitions)

['persistent://cdc/default/astrasink']


## Pulsarプロデューサー実行

実行前にサブスクリプションを作成・確認しておきます（Astraコントロールプレーンの下記パスより）。

Dashboard > Streaming (Tenants) > [利用テナント] > [Namespace and Topics]   

In [18]:
YOUR_NAME = input("your name:")

your name:taro


In [20]:
subscription_name = input("subsuctiption name:")

subsuctiption name:sub


次のプロデューサーの実行には、サブスクリプション名は不要です。
また、コンシューマー実行時に、指定したサブスクリプションがPulsarテナントに存在しない場合、自動的に作成されます。
ただし、そのサブスクリプションに送信されたメッセージの購読は、サブスクリプション作成以降に始まるので、ここでは先にサブスクリプションをテナント上で作成しました。

In [22]:
from pulsar.schema import *

class Message(Record):
    message_id = String()
    message = String()

producer = client.create_producer(topic_url,schema=JsonSchema(Message))

message = Message()

for i in range(20):
    message.message_id = str(i)
    message.message = "Hello " + YOUR_NAME + str(i)
    producer.send(message)

producer.close()

## Pulsarコンシューマー実行

In [23]:
import pulsar
import time

MODE_RUNNING = False

consumer = client.subscribe(topic_url, subscription_name)

try:
    waitingForMsg = True
    while waitingForMsg:
        try:
            msg = consumer.receive(2000)

            try:
                print("Received message '{}' id='{}'".format(msg.data(), msg.message_id()))
                # Acknowledging the message to remove from message backlog
                consumer.acknowledge(msg)
            except:
                consumer.negative_acknowledge(msg)

        except Exception as ex:
            print(ex)
            if MODE_RUNNING:
                print("Still waiting for a message...");
            else:
                print("Stop waiting for a message.");
                waitingForMsg = False
        time.sleep(1)

finally:
  consumer.close()

Received message 'b'{\n "message_id": "0",\n "message": "Hello taro0"\n}'' id='(441375,19,-1,-1)'
Received message 'b'{\n "message_id": "1",\n "message": "Hello taro1"\n}'' id='(441375,20,-1,-1)'
Received message 'b'{\n "message_id": "2",\n "message": "Hello taro2"\n}'' id='(441375,21,-1,-1)'
Received message 'b'{\n "message_id": "3",\n "message": "Hello taro3"\n}'' id='(441375,22,-1,-1)'
Received message 'b'{\n "message_id": "4",\n "message": "Hello taro4"\n}'' id='(441375,23,-1,-1)'
Received message 'b'{\n "message_id": "5",\n "message": "Hello taro5"\n}'' id='(441375,24,-1,-1)'
Received message 'b'{\n "message_id": "6",\n "message": "Hello taro6"\n}'' id='(441375,25,-1,-1)'
Received message 'b'{\n "message_id": "7",\n "message": "Hello taro7"\n}'' id='(441375,26,-1,-1)'
Received message 'b'{\n "message_id": "8",\n "message": "Hello taro8"\n}'' id='(441375,27,-1,-1)'
Received message 'b'{\n "message_id": "9",\n "message": "Hello taro9"\n}'' id='(441375,28,-1,-1)'
Received message 'b'

In [ ]:
client.close()

注意: Astraのデータベースは、トライアル利用の際に一定期間利用していない場合、休止(Hibernate)されます。Astra StreamingのSink設定は、Sink先のデータベースが一旦休止状態に入ると、データベースの再開(Resume)後にリスタートする必要があります。
この演習を、以前作成したAstra環境を使って、再実行する場合には、上記についてご注意ください。